In [1]:
import json
import pandas as pd
import numpy as np

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA
from tqdm import tqdm
import math
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score,recall_score,precision_score

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from time import gmtime, strftime

import lightgbm as lgb

import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('max_columns', 500)

In [3]:
def read_profile_data(input_dir = '../input/data_set_phase1/', profile_na_len = 67):
    profile_data = pd.read_csv(input_dir+'profiles.csv')
    profile_na = np.zeros(profile_na_len)
    profile_na[0] = -1
    profile_na = pd.DataFrame(profile_na.reshape(1, -1))
    profile_na.columns = profile_data.columns
    profile_data = profile_data.append(profile_na)
    return profile_data

In [4]:
def train_lgb(train_x, train_y, test_x, need_aug = False, need_aly = False):
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
    
    lgb_paras = {
        'objective': 'multiclass',
        'metrics': 'multiclass',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'lambda_l1': 0.01,
        'lambda_l2': 10,
        'num_class': 12,
        'seed': 2019,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'subsample':0.5,
        'colsample_bytree':0.5,
        'subsample_freq':1,
        'max_depth':-1,
        'nthread':-1,
    }
    
    #aug_cols : key is mode, val is aug rate
    aug_cols = {
        0:1,
        1:1,
        2:1,
        3:2,
        4:3,
        5:1,
        6:1,
        7:1,
        8:1,
        9:1,
        10:1,
        11:1,
    }
    
    iter_n = 50
    
    cate_cols = ['max_dist_mode', 'min_dist_mode', 'max_price_mode',
                 'min_price_mode', 'max_eta_mode', 'min_eta_mode',
                 'first_mode', 'weekday', 'hour']
    scores = []
    result_proba = []
    for fold_, (tr_idx, val_idx) in enumerate(kfold.split(train_x, train_y),1):
        print("fold: {}".format(fold_))
        tr_x, tr_y, val_x, val_y = train_x.iloc[tr_idx], train_y[tr_idx], train_x.iloc[val_idx], train_y[val_idx]
        
        # augmentation for every mode
        if need_aug:
            train_df = tr_x.copy()
            train_df['target'] = tr_y
            for key, val in aug_cols.items():
                if val > 1:
                    print("mode:{}, aug_rate:{}".format(key, val))
                    train_df = augment2(train = train_df, num_is = val, col='target', target=key)
#                     tr_x, tr_y = augment(x=tr_x.values, y=tr_y, target=key, num_is=val)
            
            tr_y = train_df['target']     
            tr_x = train_df.drop(['target'],axis=1)
            del train_df
        
        train_set = lgb.Dataset(tr_x, tr_y, categorical_feature=cate_cols)
        val_set = lgb.Dataset(val_x, val_y, categorical_feature=cate_cols)
        lgb_model = lgb.train(lgb_paras, train_set,
                              valid_sets=[val_set], early_stopping_rounds=iter_n, num_boost_round=40000, verbose_eval=iter_n, feval=eval_f)
        val_pred = np.argmax(lgb_model.predict(
            val_x, num_iteration=lgb_model.best_iteration), axis=1)
        val_score = f1_score(val_y, val_pred, average='weighted')
        result_proba.append(lgb_model.predict(
            test_x, num_iteration=lgb_model.best_iteration))
        scores.append(val_score)
        if need_aly:
            analysis_result(val_pred, val_y)
    print('cv f1-score: ', np.mean(scores))
    pred_test = np.argmax(np.mean(result_proba, axis=0), axis=1)
    return pred_test, lgb_model.feature_importance()

In [5]:
def analysis_result(pred, label):
    df_analysis = pd.DataFrame()
#     df_analysis['sid']   = data[valid_index]['sid']
    df_analysis['label'] = label
    df_analysis['pred']  = pred
    df_analysis['label'] = df_analysis['label'].astype(int)
    dic_ = df_analysis['label'].value_counts(normalize = True)
    def get_weighted_fscore(y_pred, y_true):
        f_score = 0
        for i in range(12):
            yt = y_true == i
            yp = y_pred == i
            f_score += dic_[i] * f1_score(y_true=yt, y_pred= yp)
            print(i,dic_[i],f1_score(y_true=yt, y_pred= yp), precision_score(y_true=yt, y_pred= yp),recall_score(y_true=yt, y_pred= yp))
        print(f_score)
    get_weighted_fscore(y_true =df_analysis['label'] , y_pred = df_analysis['pred'])

In [6]:
def eval_f(y_pred, train_data):
    y_true = train_data.label
    y_pred = y_pred.reshape((12, -1)).T
    y_pred = np.argmax(y_pred, axis=1)
    score = f1_score(y_true, y_pred, average='weighted')
    return 'weighted-f1-score', score, True

In [7]:
def split_train_test(data):
    train_data = data[data['click_mode'] != -1]
    test_data = data[data['click_mode'] == -1]
    submit = test_data[['sid']].copy()
    train_data = train_data.drop(['sid', 'pid'], axis=1)
    test_data = test_data.drop(['sid', 'pid'], axis=1)
#     train_data = train_data.drop(['sid'], axis=1)
#     test_data = test_data.drop(['sid'], axis=1)
    
    test_data = test_data.drop(['click_mode'], axis=1)
    train_y = train_data['click_mode'].values
    train_x = train_data.drop(['click_mode'], axis=1)
    return train_x, train_y, test_data, submit

In [8]:
def submit_result(submit, result, model_name):
    now_time = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    submit['recommend_mode'] = result
    filename = '../output/{}_result_{}.csv'.format(model_name, now_time)
    print(filename)
    submit.to_csv(filename, index=False)

In [9]:
def feature_imp(train_x, fimp):
    fi = pd.DataFrame()
    feature_name = train_x.columns
    fi['name'] = feature_name
    fi['score'] = fimp
    feature_name = fi.sort_values(by=['score'],ascending=False).head(10)['name'].values
#     fi.sort_values(by=['score'],ascending=False)
    return fi.sort_values(by=['score'],ascending=False)

In [10]:
# numpy data type aug
def augment(x=None,y=None,num_is=1,num_not=1,target=None):
    xs,xn = [],[]
    for i in range(num_not):
        mask = y!=target
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(num_is):
        mask = y==target
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [11]:
# dataframe data type aug
def augment2(train=None, num_is=1,num_p=1, col="", target=None):
    newtrain=[train]
    
#     n=train[train.target==0]
    for i in range(num_is):
        isTarget = train.loc[train[col] == target]
        newtrain.append(isTarget.apply(lambda x:x.values.take(np.random.permutation(len(isTarget)))))
    
    for i in range(num_p):
        notTarget = train.loc[train[col] != target]
        newtrain.append(notTarget.apply(lambda x:x.values.take(np.random.permutation(len(notTarget)))))
    return pd.concat(newtrain)

In [12]:
data = pd.read_pickle('../cache/basic_feats_13.pkl')
result_embedding = pd.read_pickle('../cache/embedding_plan_rank_300.pkl')
result_embedding2 = pd.read_pickle('../cache/embedding_plan_Bigram_rank_240.pkl')
profile_org_embedding = pd.read_pickle('../cache/profileORGEmbedding_svdnmflda_31.pkl')
profile_seq_embedding = pd.read_pickle('../cache/profileSeqEmbedding_svdnmflda_21.pkl')
# profile_org = read_profile_data()
plan_basic_feats = pd.read_pickle('../cache/basic_feats_58.pkl')
# plan_basic_feats2 = pd.read_pickle('../cache/statisitc_rank_feats_37.pkl')
# deepwalk_alpha_df = pd.read_pickle('../cache/deepwalk_alpha_10.pkl')


data = pd.concat([data, result_embedding], axis=1)
data = pd.concat([data, result_embedding2], axis=1)
data = data.merge(profile_org_embedding, on='pid', how='left')
data = data.merge(profile_seq_embedding, on='pid', how='left')
# data = data.merge(profile_org, on='pid', how='left')
data = data.merge(plan_basic_feats, on='sid', how='left')
# data = data.merge(plan_basic_feats2.drop(['pid','click_mode'],axis=1), on='sid', how='left')
# data = data.merge(deepwalk_alpha_df, on='sid', how='left')

data.head()

,click_mode,pid,sid,o1,o2,d1,d2,weekday,hour,L1Dis,L2Dis,CheDis,CosDis,svd_mode_texts_0,svd_mode_texts_1,svd_mode_texts_2,svd_mode_texts_3,svd_mode_texts_4,svd_mode_texts_5,svd_mode_texts_6,svd_mode_texts_7,svd_mode_texts_8,svd_mode_texts_9,nmf_mode_texts_0,nmf_mode_texts_1,nmf_mode_texts_2,nmf_mode_texts_3,nmf_mode_texts_4,nmf_mode_texts_5,nmf_mode_texts_6,nmf_mode_texts_7,nmf_mode_texts_8,nmf_mode_texts_9,svd_distance_texts_0,svd_distance_texts_1,svd_distance_texts_2,svd_distance_texts_3,svd_distance_texts_4,svd_distance_texts_5,svd_distance_texts_6,svd_distance_texts_7,svd_distance_texts_8,svd_distance_texts_9,nmf_distance_texts_0,nmf_distance_texts_1,nmf_distance_texts_2,nmf_distance_texts_3,nmf_distance_texts_4,nmf_distance_texts_5,nmf_distance_texts_6,nmf_distance_texts_7,nmf_distance_texts_8,nmf_distance_texts_9,svd_price_texts_0,svd_price_texts_1,svd_price_texts_2,svd_price_texts_3,svd_price_texts_4,svd_price_texts_5,svd_price_texts_6,svd_price_texts_7,svd_price_texts_8,svd_price_texts_9,nmf_price_texts_0,nmf_price_texts_1,nmf_price_texts_2,nmf_price_texts_3,nmf_price_texts_4,nmf_price_texts_5,nmf_price_texts_6,nmf_price_texts_7,nmf_price_texts_8,nmf_price_texts_9,svd_eta_texts_0,svd_eta_texts_1,svd_eta_texts_2,svd_eta_texts_3,svd_eta_texts_4,svd_eta_texts_5,svd_eta_texts_6,svd_eta_texts_7,svd_eta_texts_8,svd_eta_texts_9,nmf_eta_texts_0,nmf_eta_texts_1,nmf_eta_texts_2,nmf_eta_texts_3,nmf_eta_texts_4,nmf_eta_texts_5,nmf_eta_texts_6,nmf_eta_texts_7,nmf_eta_texts_8,nmf_eta_texts_9,svd_price_mode_texts_0,svd_price_mode_texts_1,svd_price_mode_texts_2,svd_price_mode_texts_3,svd_price_mode_texts_4,svd_price_mode_texts_5,svd_price_mode_texts_6,svd_price_mode_texts_7,svd_price_mode_texts_8,svd_price_mode_texts_9,nmf_price_mode_texts_0,nmf_price_mode_texts_1,nmf_price_mode_texts_2,nmf_price_mode_texts_3,nmf_price_mode_texts_4,nmf_price_mode_texts_5,nmf_price_mode_texts_6,nmf_price_mode_texts_7,nmf_price_mode_texts_8,nmf_price_mode_texts_9,svd_distance_mode_texts_0,svd_distance_mode_texts_1,svd_distance_mode_texts_2,svd_distance_mode_texts_3,svd_distance_mode_texts_4,svd_distance_mode_texts_5,svd_distance_mode_texts_6,svd_distance_mode_texts_7,svd_distance_mode_texts_8,svd_distance_mode_texts_9,nmf_distance_mode_texts_0,nmf_distance_mode_texts_1,nmf_distance_mode_texts_2,nmf_distance_mode_texts_3,nmf_distance_mode_texts_4,nmf_distance_mode_texts_5,nmf_distance_mode_texts_6,nmf_distance_mode_texts_7,nmf_distance_mode_texts_8,nmf_distance_mode_texts_9,svd_eta_mode_texts_0,svd_eta_mode_texts_1,svd_eta_mode_texts_2,svd_eta_mode_texts_3,svd_eta_mode_texts_4,svd_eta_mode_texts_5,svd_eta_mode_texts_6,svd_eta_mode_texts_7,svd_eta_mode_texts_8,svd_eta_mode_texts_9,nmf_eta_mode_texts_0,nmf_eta_mode_texts_1,nmf_eta_mode_texts_2,nmf_eta_mode_texts_3,nmf_eta_mode_texts_4,nmf_eta_mode_texts_5,nmf_eta_mode_texts_6,nmf_eta_mode_texts_7,nmf_eta_mode_texts_8,nmf_eta_mode_texts_9,svd_dis_pri_eta_mode_texts_0,svd_dis_pri_eta_mode_texts_1,svd_dis_pri_eta_mode_texts_2,svd_dis_pri_eta_mode_texts_3,svd_dis_pri_eta_mode_texts_4,svd_dis_pri_eta_mode_texts_5,svd_dis_pri_eta_mode_texts_6,svd_dis_pri_eta_mode_texts_7,svd_dis_pri_eta_mode_texts_8,svd_dis_pri_eta_mode_texts_9,nmf_dis_pri_eta_mode_texts_0,nmf_dis_pri_eta_mode_texts_1,nmf_dis_pri_eta_mode_texts_2,nmf_dis_pri_eta_mode_texts_3,nmf_dis_pri_eta_mode_texts_4,nmf_dis_pri_eta_mode_texts_5,nmf_dis_pri_eta_mode_texts_6,nmf_dis_pri_eta_mode_texts_7,nmf_dis_pri_eta_mode_texts_8,nmf_dis_pri_eta_mode_texts_9,svd_rank_mode_texts_0,svd_rank_mode_texts_1,svd_rank_mode_texts_2,svd_rank_mode_texts_3,svd_rank_mode_texts_4,svd_rank_mode_texts_5,svd_rank_mode_texts_6,svd_rank_mode_texts_7,svd_rank_mode_texts_8,svd_rank_mode_texts_9,nmf_rank_mode_texts_0,nmf_rank_mode_texts_1,nmf_rank_mode_texts_2,nmf_rank_mode_texts_3,nmf_rank_mode_texts_4,nmf_rank_mode_texts_5,nmf_rank_mode_texts_6,nmf_rank_mode_texts_7,nmf_rank_mode_texts_8,nmf_rank_mode_texts_9,svd_rank_distance_texts_0,svd_rank_distance_texts_1,svd_rank_distance

In [13]:
train_x, train_y, test_x, submit = split_train_test(data)

In [14]:
result_lgb, fimp = train_lgb(train_x, train_y, test_x, need_aug=True, need_aly = True)

fold: 1
mode:3, aug_rate:2
mode:4, aug_rate:3
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 1.07896	valid_0's weighted-f1-score: 0.670795
[100]	valid_0's multi_logloss: 0.981325	valid_0's weighted-f1-score: 0.674659
[150]	valid_0's multi_logloss: 0.956963	valid_0's weighted-f1-score: 0.675915
[200]	valid_0's multi_logloss: 0.94757	valid_0's weighted-f1-score: 0.676824
[250]	valid_0's multi_logloss: 0.942726	valid_0's weighted-f1-score: 0.677275
[300]	valid_0's multi_logloss: 0.93953	valid_0's weighted-f1-score: 0.677249
Early stopping, best iteration is:
[279]	valid_0's multi_logloss: 0.940772	valid_0's weighted-f1-score: 0.677447
0 0.09332533373331334 0.33100026579250463 0.9559877175025588 0.20015000535733418
1 0.1407329633518324 0.6737540672014433 0.6163140213355337 0.7430012789540997
2 0.2729763511824409 0.8814504881450488 0.840956721333289 0.9260412469321221
3 0.04925753712314384 0.15360648076024302 0.3302076356329538 0.1000812017864393

8 0.0037102226133568015 0.35213204951856947 0.3595505617977528 0.3450134770889488
9 0.0977258635518131 0.7294614960485205 0.6619412941961308 0.8123209169054442
10 0.029761785707142427 0.5479526595021086 0.46034285714285716 0.676747311827957
11 0.01217073024381463 0.6112841813776394 0.52811004784689 0.7255546425636812
0.6774884638799831
cv f1-score:  0.6772350777588187


In [15]:
#mode 3 aug2, mode 4 aug3
feature_imp(train_x, fimp)

,name,score
5,hour,4757
630,first_mode,3100
72,svd_eta_texts_2,1692
172,svd_rank_mode_texts_2,1417
73,svd_eta_texts_3,1299
84,nmf_eta_texts_4,1279
171,svd_rank_mode_texts_1,1133
71,svd_eta_texts_1,1060
623,std_eta,986
173,svd_rank_mode_texts_3,978


In [18]:
# mode 3 aug2, mode 4 aug2
feature_imp(train_x, fimp)

,name,score
5,hour,2218
630,first_mode,2188
72,svd_eta_texts_2,1189
660,transport_mode_lst_deepwalk_3,1024
172,svd_rank_mode_texts_2,997
84,nmf_eta_texts_4,858
73,svd_eta_texts_3,810
657,transport_mode_lst_deepwalk_0,807
71,svd_eta_texts_1,778
663,transport_mode_lst_deepwalk_6,770


In [14]:
result_lgb2, fimp2 = train_lgb(train_x, train_y, test_x, need_aug=False, need_aly = True)

fold: 1
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 1.01518	valid_0's weighted-f1-score: 0.670555
[100]	valid_0's multi_logloss: 0.922786	valid_0's weighted-f1-score: 0.673729
[150]	valid_0's multi_logloss: 0.901949	valid_0's weighted-f1-score: 0.675011
[200]	valid_0's multi_logloss: 0.896046	valid_0's weighted-f1-score: 0.675591
[250]	valid_0's multi_logloss: 0.894185	valid_0's weighted-f1-score: 0.675322
Early stopping, best iteration is:
[200]	valid_0's multi_logloss: 0.896046	valid_0's weighted-f1-score: 0.675591
0 0.09332533373331334 0.33315733896515315 0.9320531757754801 0.2028286724525876
1 0.1407329633518324 0.6752068998319434 0.609689056863082 0.7565013500071053
2 0.2729763511824409 0.8827821011673153 0.8394727278734019 0.9308033261291623
3 0.04925753712314384 0.11558053750219568 0.42894393741851367 0.06678846934632562
4 0.025218739063046846 0.01405152224824356 0.45 0.007137192704203013
5 0.09495525223738813 0.833007430582714 0.7

In [15]:
feature_imp(train_x, fimp2)

,name,score
5,hour,5170
630,first_mode,991
1,o2,828
0,o1,803
2,d1,663
3,d2,654
623,std_eta,637
581,svd_pro_seq_1,524
578,nmf_pro_org_8,484
4,weekday,404


In [16]:
submit_result(submit, result_lgb, 'lgb')

../output/lgb_result_2019-05-12-15-44-58.csv
